In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix,log_loss, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from imblearn.over_sampling import RandomOverSampler
import nltk
import gensim
import os
import matplotlib.pyplot as plt
import re

In [59]:
data = pd.read_csv('PS3_training_data.txt',  delimiter='	', names = ["index","sentence","polarity","issue","genre", "uname"])
data.head()

,index,sentence,polarity,issue,genre,uname
0,0,This is definitely a must have if your state d...,POSITIVE,NONE,GENRE_B,NaN
1,1,It's a great place and I highly recommend it.,POSITIVE,NONE,GENRE_B,NaN
2,2,"I will see the doctors, take some blood tests ...",NEUTRAL,GOING_TO_PLACES,GENRE_A,NaN
3,3,I can tell you about having my phone and elect...,NEGATIVE,MONEY_ISSUE,GENRE_A,NaN
4,4,Their steaks are 100% recommended!,POSITIVE,NONE,GENRE_B,NaN


In [60]:
data = data[["sentence","polarity","issue","genre"]]
# hist = data.hist(bin = 3)
data.head()


,sentence,polarity,issue,genre
0,This is definitely a must have if your state d...,POSITIVE,NONE,GENRE_B
1,It's a great place and I highly recommend it.,POSITIVE,NONE,GENRE_B
2,"I will see the doctors, take some blood tests ...",NEUTRAL,GOING_TO_PLACES,GENRE_A
3,I can tell you about having my phone and elect...,NEGATIVE,MONEY_ISSUE,GENRE_A
4,Their steaks are 100% recommended!,POSITIVE,NONE,GENRE_B


In [61]:
POS = len(data[data.polarity == "POSITIVE"])
NEG = len(data[data.polarity == "NEGATIVE"])
NEU = len(data[data.polarity == "NEUTRAL"])
total = len(data)
print(" POS : % , NEG : %, NEU : %",(POS/total,NEG/total,NEU/total))

 POS : % , NEG : %, NEU : % (0.41953125, 0.50078125, 0.0796875)


In [63]:
def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """
    
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text

In [64]:
cv = CountVectorizer(stop_words="english",
                             preprocessor=clean_text).fit(data["sentence"])
df_train,df_test = train_test_split(data,test_size=0.2, random_state=35)



In [65]:
# x_train = mean_embedding_vectorizer.fit_transform(df_train['sentence'])
# y_train = df_train['polarity']
# x_test = mean_embedding_vectorizer.fit_transform(df_test['sentence'])
# y_test = df_test['polarity']
# print((x_train.shape,y_train.shape))
# ros = RandomOverSampler(random_state=42)
# x_train,y_train =  ros.fit_resample(x_train,y_train)
# print((x_train.shape,y_train.shape))

In [77]:
x_train = cv.transform(df_train['sentence'])
y_train = df_train['polarity']
# y_train = df_train['genre']
# y_train = df_train['issue']



x_test = cv.transform(df_test['sentence']) 
y_test = df_test['polarity']
# y_test = df_test['genre']
# y_test = df_test['issue']



print((x_train.shape,y_train.shape))
# ros = RandomOverSampler(random_state=42)
# x_train,y_train =  ros.fit_resample(x_train,y_train)
# print((x_train.shape,y_train.shape))

((2048, 6491), (2048,))


In [78]:
def generate_report(model,x_test,y_test):
    predict = model.predict(x_test)
#     print(predict.shape)
#     print(y_test.shape)
    print(accuracy_score(y_test, predict))
    print(classification_report(y_test, predict))
models = [
#     RandomForestClassifier(),
    LinearSVC(),
    GradientBoostingClassifier(),
    LogisticRegression()
    
     
]
for model in models:
    model.fit(x_train,y_train)
    generate_report(model,x_test,y_test)



0.6953125
              precision    recall  f1-score   support

    NEGATIVE       0.71      0.79      0.75       256
     NEUTRAL       0.50      0.26      0.34        47
    POSITIVE       0.70      0.67      0.69       209

   micro avg       0.70      0.70      0.70       512
   macro avg       0.64      0.57      0.59       512
weighted avg       0.69      0.70      0.69       512

0.67578125
              precision    recall  f1-score   support

    NEGATIVE       0.64      0.93      0.76       256
     NEUTRAL       0.62      0.17      0.27        47
    POSITIVE       0.78      0.48      0.60       209

   micro avg       0.68      0.68      0.68       512
   macro avg       0.68      0.53      0.54       512
weighted avg       0.70      0.68      0.65       512

0.705078125
              precision    recall  f1-score   support

    NEGATIVE       0.69      0.81      0.75       256
     NEUTRAL       0.79      0.23      0.36        47
    POSITIVE       0.72      0.68      0.7